In [18]:
from dateutil.relativedelta import *
import statsmodels.api as sm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame, Series
import math
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
users={
    'A': {1: 55.7837159,  2: 38.79102923,  3: 53.06062927, 4: 77.35448691,  5: 46.24008587,  6: 62.22685343,  7: 26.45137724,  8: 72.01672276,
          9: 52.37349205, 10: 29.82266582, 11: 0.0,        12: 45.8980471,  13: 48.19106369, 14: 63.35712229, 15: 23.21661838, 16: 59.44735127,
          17: 0.0,        18: 45.89132096, 19: 51.833334,  20: 34.50565703, 21: 42.07319553, 22: 49.72173371, 23: 37.24038386, 24: 61.07287597,
          25: 31.95258913,26: 49.77664266, 27: 34.26467847,28: 40.42737935, 29: 39.52368174, 30: 63.61050253},
    'B': {1: 53.92138433, 2: 75.96895801,  3: 47.3358618,  4: 34.43917074,  5: 31.73398622,  6: 37.27441235,  7: 51.46850643,  8: 62.11333106,
          9: 46.42235397, 10: 0.0,         11: 64.9147645, 12: 31.18722915, 13: 44.40628842, 14: 49.1348466,  15: 0.0,         16: 44.1748031,
          17: 30.78111034,18: 45.09351328, 19: 56.98644867,20: 66.11121943, 21: 56.30083081, 22: 50.83298741, 23: 0.0,         24: 40.57205673,
          25: 74.29451295,26: 51.49535171, 27: 55.35656942,28: 51.66820241, 29: 0.0,         30: 0.0,         31: 69.28899667},
    'C': {1: 0.0,         2: 0.0,          3: 40.3796516,  4: 74.32577092,  5: 60.72803398,  6: 0.0,          7: 34.11217187,  8: 0.0,
          9: 41.59839299, 10: 0.0,         11: 34.3265856, 12: 0.0,         13: 0.0,         14: 55.84045123, 15: 0.0,         16: 32.75655016,
          17: 0.0,        18: 61.44115687, 19: 0.0,        20: 60.27528438, 21: 46.3705737,  22: 69.17061219, 23: 0.0,         24: 0.0,
          25: 32.07981437,26: 55.9164317,  27: 59.51761531,28: 42.40801701, 29: 0.0,         30: 0.0,         31: 0.0},
    'D': {1: 64.8789546,  2: 60.64071999,  3: 58.78405112, 4: 50.78904116,  5: 40.11804384,  6: 48.74361854,  7: 45.97086653,  8: 68.02961542,
          9: 56.01684064, 10: 35.31178863, 11: 52.16548218,12: 59.81178823, 13: 49.09581183, 14: 49.92630457, 15: 38.91152044, 16: 53.47773179,
          17: 40.14402742,18: 52.08754281, 19: 55.32213754,20: 46.36922738, 21: 39.39928903, 22: 47.74161219, 23: 42.49630517, 24: 57.18356273,
          25: 62.45974214,26: 41.41548008, 27: 0.0,        28: 38.23049652, 29: 48.7694508,  30: 41.63833288, 31: 61.31450282},
    'E': {1: 39.55841581, 2: 0.0,          3: 51.73960675, 4: 47.39950138,  5: 58.83767412,  6: 44.8788159,   7: 35.67813884,  8: 64.65389258,
          9: 46.7520161,  10: 36.5959086,  11: 32.59533141,12: 50.68200521,13: 38.16494319,  14: 47.5194743,  15: 0.0,         16: 66.05813146,
          17: 0.0,        18: 66.40272557, 19: 45.04071136,20: 35.62896721,21: 36.31338687,  22: 43.46422358, 23: 0.0,         24: 35.91314858,
          25: 47.7794358, 26: 56.29132847, 27: 38.68296878,28: 28.79842199,29: 0.0,          30: 39.15472954, 31: 40.19794217},
    'F': {1: 52.66390841, 2: 55.73126806,  3: 0.0,         4: 59.58952645, 5: 50.36684938,   6: 56.9879805,   7: 23.74010684,  8: 76.89137015,
          9: 0.0,         10: 35.50775387, 11: 0.0,        12: 0.0,        13: 61.10876092,  14: 69.50860227, 15: 26.97991769, 16: 54.33200857,
          17: 0.0,        18: 48.36182221, 19: 0.0,        20: 53.03029787,21: 34.84530004,  22: 31.06643584, 23: 50.77266556, 24: 53.68593693,
          25: 43.31794922,26: 42.36998142, 27: 43.3931631, 28: 33.26626807,29: 58.60551186,  30: 63.66106261, 31: 48.52936174},
    'G': {1: 52.2395188,  2: 45.12926373,  3: 68.82995251, 4: 65.29121691, 5: 60.74877538,   6: 45.37592309,  7: 23.31263625,  8: 70.95384465,
          9: 50.82941181, 10: 51.74951176, 11: 32.89123617,12: 64.20212941,13: 46.93123685,  14: 69.65797247, 15: 35.16343744, 16: 71.41280356,
          17: 70.87628992,18: 50.59184751, 19: 35.84594525,20: 31.99846196,21: 41.95518218,  22: 38.59874603, 23: 46.01653548, 24: 71.81677851,
          25: 35.12160389,26: 48.10862861, 27: 27.06170034,28: 25.19386059,29: 40.35835709,  30: 61.90667658, 31: 42.70146579},
    'H': {1: 48.53654137, 2: 39.24109587,  3: 53.39570651, 4: 64.14179565, 5: 55.9746734,    6: 48.83162846,  7: 36.90986023,  8: 69.46714131,
          9: 53.04368696, 10: 30.25569233, 11: 31.47173609,12: 69.02862625,13: 40.00351192,  14: 63.25594312, 15: 32.55139598, 16: 68.1643389,
          17: 58.3490784, 18: 59.54860223, 19: 42.20220319,20: 27.13117008,21: 41.2448021,   22: 37.95987902, 23: 49.41823313, 24: 64.78335108,
          25: 45.43301161,26: 53.18277312, 27: 26.78176805,28: 34.1507348, 29: 29.06617622,  30: 56.23538877, 31: 35.13479673},
    'I': {1: 59.54088817, 2: 59.8667084,   3: 31.68472897, 4: 45.82077784, 5: 44.65309958,   6: 66.98114916,  7: 38.88837935,  8: 67.7358788,
          9: 48.67881181, 10: 19.95730139, 11: 40.21465042,12: 0.0,        13: 46.19292264,  14: 45.6920564,  15: 43.1580082,  16: 56.13166517,
          17: 45.87437905,18: 34.88736273, 19: 0.0,        20: 55.34414956,21: 42.46803025,  22: 35.23362492, 23: 35.2570074,  24: 46.31324111,
          25: 52.11203162,26: 46.29124183, 27: 45.24423358,28: 46.7731166, 29: 48.82397725,  30: 45.0544841,  31: 65.71047992},
    'J': {1: 61.96394674, 2: 67.41091208,  3: 42.54274102, 4: 44.81151314, 5: 43.03513942,   6: 40.45838483,  7: 33.47668727,  8: 75.49636161,
          9: 49.95923103, 10: 0.0,         11: 54.91598853,12: 57.04622433,13: 0.0,          14: 53.66144451, 15: 0.0,         16: 48.06678561,
          17: 40.94313695,18: 54.53439811, 19: 54.22228739,20: 54.95678304,21: 36.97964901,  22: 43.67253161, 23: 0.0,         24: 39.02529432,
          25: 51.12864057,26: 44.05541123, 27: 34.43817655,28: 42.36545444,29: 0.0,          30: 0.0,         31: 60.05840386},
    'K': {1: 51.70699274, 2: 0.0,          3: 54.08718953, 4: 42.76885392, 5: 44.36777271,   6: 31.2847248,   7: 37.56266998,  8: 0.0,
          9: 48.89505879, 10: 25.62840078, 11: 0.0,        12: 0.0,        13: 0.0,          14: 70.17172065, 15: 51.71349843, 16: 0.0,
          17: 52.49962545,18: 72.20820022, 19: 0.0,        20: 44.6546433, 21: 39.41387276,  22: 35.10406312, 23: 50.74975178, 24: 64.06776636,
          25: 0.0,        26: 40.04768227, 27: 0.0,        28: 30.21619813,29: 55.25783948,  30: 53.42127162, 31: 59.31398425},
    'L': {1: 52.10410172, 2: 60.04901268,  3: 39.96063404, 4: 30.44631192, 5: 51.53953088,   6: 33.08363554,  7: 24.28967773,  8: 54.64085812,
          9: 47.82662348, 10: 35.07622403, 11: 44.79621102,12: 59.96732082,13: 0.0,          14: 65.2546536,  15: 50.18427406, 16: 67.63642348,
          17: 35.46781871,18: 50.04389968, 19: 29.92358141,20: 53.523722,  21: 25.81641837,  22: 0.0,         23: 53.60201674, 24: 43.11948702,
          25: 64.38694856,26: 42.34902778, 27: 0.0,        28: 40.3380021, 29: 68.2479411,   30: 64.2270816,  31: 69.28812642},
    'M': {1: 46.95464656, 2: 43.66808696,  3: 45.60385109, 4: 67.56927393, 5: 60.12803344,   6: 56.25759863,  7: 29.37769812,  8: 75.04979358,
          9: 44.35518755, 10: 36.95088358, 11: 37.89630465,12: 50.01181347,13: 36.21742876,  14: 51.51523362, 15: 33.09510878, 16: 57.52576344,
          17: 64.66442699,18: 57.21938676, 19: 54.79125841,20: 32.39684961,21: 50.92386553,  22: 49.20829529, 23: 47.70141619, 24: 44.94328092,
          25: 44.04004755,26: 58.29959786, 27: 36.95022728,28: 27.20201223,29: 41.82091873,  30: 49.97621123, 31: 50.15028845},
    'N': {1: 44.56337944, 2: 66.24849102,  3: 61.23835787, 4: 45.92906158, 5: 51.18346942,   6: 31.94630715,  7: 38.66650296,  8: 54.65721057,
          9: 42.44007662, 10: 0.0,         11: 49.92077329,12: 47.92271932,13: 52.77558343,  14: 56.00711003, 15: 0.0,         16: 62.91436227,
          17: 54.25963543,18: 0.0,         19: 40.02949988,20: 47.19285647,21: 37.89704332,  22: 35.46821707, 23: 0.0,         24: 50.53346133,
          25: 49.76398805,26: 38.30163433, 27: 0.0,        28: 25.90265432,29: 0.0,          30: 0.0,         31: 42.08722438},
    'O': {1: 73.57869609, 2: 50.14836781,  3: 0.0,         4: 0.0,         5: 50.89870023,   6: 51.43007163,  7: 49.91460251,  8: 83.32788306,
          9: 0.0,         10: 0.0,         11: 42.61515519,12: 63.42639576,13: 0.0,          14: 66.83614053, 15: 0.0,         16: 45.04828393,
          17: 62.79114956,18: 0.0,         19: 30.36077788,20: 68.88669982,21: 0.0,          22: 0.0,         23: 0.0,         24: 55.3846227,
          25: 65.35981429,26: 44.54460948, 27: 0.0,        28: 54.19074989,29: 0.0,          30: 0.0,         31: 70.0390869},
    'P': {1: 44.5549775,  2: 0.0,          3: 40.4599677,  4: 62.47968183, 5: 61.71082863,   6: 0.0,          7: 19.30418741,  8: 60.23771341,
          9: 40.06110428, 10: 50.5199446,  11: 34.32533318,12: 53.89601449,13: 40.18995177,  14: 46.82660308, 15: 30.97267369, 16: 57.46619098,
          17: 67.03239421,18: 68.01033139, 19: 46.90212603,20: 31.68347191,21: 46.42522442,  22: 41.16216481, 23: 61.62044265, 24: 38.06305387,
          25: 38.94866645,26: 43.09916702, 27: 35.84273691,28: 28.94878864,29: 38.16331406,  30: 47.36752439, 31: 47.98505358},
    'Q': {1: 53.39124121, 2: 41.57467791,  3: 63.16234551 ,4: 53.89953359, 5: 55.29717996,   6: 34.49614376,  7: 37.82680012,  8: 67.07059777,
          9: 52.45512701, 10: 35.91122967, 11: 38.04955164,12: 0.0,        13: 44.89707677,  14: 48.15358126, 15: 34.06629576, 16: 63.07103108,
          17: 60.75368538,18: 61.49142144, 19: 0.0,        20: 42.007612,  21: 47.03026417,  22: 55.44097904, 23: 57.6516194,  24: 54.78493124,
          25: 58.99156597,26: 60.12529923, 27: 57.25075379,28: 48.32864844,29: 45.43988551,  30: 71.36990438, 31: 54.44649705},
    'R': {1: 67.02574585, 2: 0.0,          3: 50.10435977, 4: 44.15513925, 5: 0.0,           6: 45.71626622,  7: 56.50316019,  8: 69.16882196,
          9: 65.48316262, 10: 31.25366179, 11: 0.0,        12: 51.24080338,13: 46.22165805,  14: 0.0,         15: 0.0,         16: 0.0,
          17: 0.0,        18: 55.89264212, 19: 44.86111818,20: 53.97798918,21: 36.79380983,  22: 33.51013822, 23: 0.0,         24: 41.96188927,
          25: 0.0,        26: 52.08412673, 27: 0.0,        28: 48.16509687,29: 0.0,          30: 39.35983209, 31: 58.50440215},
    'S': {1: 58.52298423, 2: 39.43737542,  3: 40.65016313, 4: 59.48674382, 5: 35.07693108,   6: 78.7696149,   7: 35.58590092,  8: 63.26159593,
          9: 64.00595746, 10: 0.0,         11: 47.33576136,12: 51.27380893,13: 0.0,          14: 50.4325614,  15: 0.0,         16: 57.06449411,
          17: 36.83979383,18: 45.4354308,  19: 34.78124215,20: 46.78545015,21: 40.83768305,  22: 40.76427663, 23: 0.0,         24: 55.30956069,
          25: 35.65786787,26: 30.66724534, 27: 32.88579174,28: 48.31274901,29: 0.0,          30: 0.0,         31: 53.09316373},
    'T': {1: 60.89052918, 2: 47.33625739,  3: 60.51040708, 4: 69.1300208,  5: 42.75768512,   6: 54.78981822,  7: 27.37731178,  8: 67.53923028,
          9: 58.22000917, 10: 29.16377866, 11: 38.81771476,12: 59.13803121,13: 0.0,          14: 61.21839558, 15: 37.24057715, 16: 63.41802569,
          17: 61.74769626,18: 40.36106714, 19: 34.45135029,20: 42.50671412,21: 41.28563206,  22: 38.14529041, 23: 24.32100874, 24: 58.06399592,
          25: 37.99160354,26: 40.03589055, 27: 27.31603488,28: 24.66543563,29: 30.65414342,  30: 62.00175987, 31: 53.97149098},
    'U': {1: 50.02448926, 2: 58.09093937,  3: 64.72806775, 4: 49.71679509, 5: 49.76952005,   6: 42.99888813,  7: 46.05368118,  8: 54.05803601,
          9: 46.3743675,  10: 49.93905157, 11: 57.51501383,12: 68.59842503,13: 38.09670627,  14: 53.58504869, 15: 38.80140373, 16: 70.75992487,
          17: 53.52274017,18: 56.33412284, 19: 30.68047923,20: 36.34075485,21: 33.24206275,  22: 23.32383843, 23: 38.58958009, 24: 52.52129479,
          25: 37.31398614,26: 34.72779412, 27: 20.72163932,28: 13.40905939,29: 28.34894587,  30: 37.02779941, 31: 41.76372954},
    'V': {1: 66.89466712, 2: 0.0,          3: 42.4279984,  4: 59.79168963, 5: 38.31619639,   6: 62.88111571,  7: 27.79607121,  8: 0.0,
          9: 61.8443124,  10: 32.61483159, 11: 34.3232217, 12: 0.0,        13: 0.0,          14: 64.83321087, 15: 30.74506657, 16: 50.918534,
          17: 48.42797225,18: 37.45537259, 19: 0.0,        20: 45.5451389, 21: 33.9318992,   22: 29.90054828, 23: 24.79563745, 24: 64.67970728,
          25: 37.42146692,26: 41.8824673,  27: 29.63881277,28: 24.15222884,29: 44.44116257,  30: 51.57523731, 31: 55.96324898},
    'W': {1: 40.15234415, 2: 39.45811281,  3: 55.08745317, 4: 66.38510583, 5: 35.54347121,   6: 73.71217189,  7: 30.17730921,  8: 59.78685891,
          9: 47.45136575, 10: 14.1159019,  11: 35.49816309,12: 48.25546618,13: 34.49983204,  14: 51.28322114, 15: 21.07844673, 16: 49.03549704,
          17: 42.69171929,18: 33.0137052,  19: 40.37534357,20: 20.71014538,21: 47.08569467,  22: 44.07493456, 23: 17.17678472, 24: 47.81032906,
          25: 33.25789859,26: 47.62199468, 27: 23.15151998,28: 14.33703651,29: 16.54880216,  30: 48.84033475, 31: 30.01853393},
    'X': {1: 72.28220368, 2: 55.78240836,  3: 58.68545687, 4: 55.49870849, 5: 39.42439451,   6: 60.02105355,  7: 48.82801468,  8: 74.73575491,
          9: 68.7528266,  10: 17.08374125, 11: 51.17714558,12: 50.42144623,13: 69.36283648,  14: 67.64241162, 15: 32.85783722, 16: 52.72003173,
          17: 24.32763957,18: 47.72284562, 19: 37.11668926,20: 44.55814377,21: 41.66878306,  22: 47.18495954, 23: 37.59932659, 24: 59.02162101,
          25: 43.62188536,26: 37.04562531, 27: 43.46727736,28: 32.28214244,29: 27.62238011,  30: 49.16765004, 31: 48.65932658},
    'Y': {1: 54.12448438, 2: 58.62879734,  3: 56.04957136, 4: 53.36707586, 5: 36.797225,     6: 65.26809456,  7: 38.0304097,   8: 74.41154458,
          9: 45.40693858, 10: 23.28221467, 11: 47.34446321,12: 55.08407807,13: 39.51140596,  14: 52.94905288, 15: 31.40224066, 16: 48.85303491,
          17: 47.39839643,18: 52.97734628, 19: 50.35336019,20: 39.80308324,21: 48.01041741,  22: 44.42840662, 23: 28.03575359, 24: 49.522199,
          25: 45.13051768,26: 54.04550016, 27: 0.0,        28: 21.00784375,29: 40.05638283,  30: 49.83801354, 31: 53.4858192},
    'Z': {1: 51.42091269, 2: 70.04292197,  3: 38.95839737, 4: 59.61161322, 5: 34.14414949,   6: 62.93881472,  7: 48.18047234,  8: 81.09653884,
          9: 43.54536777, 10: 34.52791036, 11: 0.0,        12: 49.4460391, 13: 39.7939887,   14: 48.57569659, 15: 44.83250392, 16: 49.69867529,
          17: 36.21365656,18: 50.06416461, 19: 48.57553701,20: 48.76032977,21: 52.497931,    22: 48.17159753, 23: 13.86612202, 24: 34.61310738,
          25: 54.98051158,26: 52.93722955, 27: 46.52222948,28: 37.68730816,29: 40.51870481,  30: 52.0309195,  31: 42.13502479}
}

In [20]:
items={
    1:  {'A': 55.7837159, 'B': 53.92138433, 'C': 0.0,         'D': 64.8789546, 'E': 39.55841581, 'F': 52.66390841, 'G': 52.2395188, 
         'H': 48.53654137,'I': 59.54088817, 'J': 61.96394674, 'K': 51.70699274,'L': 52.10410172, 'M': 46.95464656, 'N': 44.56337944,
         'O': 73.57869609,'P': 44.5549775,  'Q': 53.39124121, 'R': 67.02574585,'S': 58.52298423, 'T': 60.89052918, 'U': 50.02448926,
         'V': 66.89466712,'W': 40.15234415, 'X': 72.28220368, 'Y': 54.12448438,'Z': 51.42091269},
    2:  {'A': 38.79102923,'B': 75.96895801, 'C': 0.0,         'D': 60.64071999,'E': 0.0,         'F': 55.73126806, 'G': 45.12926373,
         'H': 39.24109587,'I': 59.8667084,  'J': 67.41091208, 'K': 0.0,        'L': 60.04901268, 'M': 43.66808696, 'N': 66.24849102,
         'O': 50.14836781,'P': 0.0,         'Q': 41.57467791, 'R': 0.0,        'S': 39.43737542, 'T': 47.33625739, 'U': 58.09093937,
         'V': 0.0,        'W': 39.45811281, 'X': 55.78240836, 'Y': 58.62879734,'Z': 70.04292197},
    3:  {'A': 53.06062927,'B': 47.3358618,  'C': 40.3796516,  'D': 58.78405112,'E': 51.73960675, 'F': 0.0,         'G': 68.82995251,
         'H': 53.39570651,'I': 31.68472897, 'J': 42.54274102, 'K': 54.08718953,'L': 39.96063404, 'M': 45.60385109, 'N': 61.23835787,
         'O': 0.0,        'P': 40.4599677,  'Q': 63.16234551, 'R': 50.10435977,'S': 40.65016313, 'T': 60.51040708, 'U': 64.72806775,
         'V': 42.4279984, 'W': 55.08745317, 'X': 58.68545687, 'Y': 56.04957136,'Z': 38.95839737},
    4:  {'A': 77.35448691,'B': 34.43917074, 'C': 74.32577092, 'D': 50.78904116,'E': 47.39950138, 'F': 59.58952645, 'G': 65.29121691,
         'H': 64.14179565,'I': 45.82077784, 'J': 44.81151314, 'K': 42.76885392,'L': 30.44631192, 'M': 67.56927393, 'N': 45.92906158,
         'O': 0.0,        'P': 62.47968183, 'Q': 53.89953359, 'R': 44.15513925,'S': 59.48674382, 'T': 69.1300208,  'U': 49.71679509,
         'V': 59.79168963,'W': 66.38510583, 'X': 55.49870849, 'Y': 53.36707586,'Z': 59.61161322},
    5:  {'A': 46.24008587,'B': 31.73398622, 'C': 60.72803398, 'D': 40.11804384,'E': 58.83767412, 'F': 50.36684938, 'G': 60.74877538,
         'H': 55.9746734, 'I': 44.65309958, 'J': 43.03513942, 'K': 44.36777271,'L': 51.53953088, 'M': 60.12803344, 'N': 51.18346942,
         'O': 50.89870023,'P': 61.71082863, 'Q': 55.29717996, 'R': 0.0,        'S': 35.07693108, 'T': 42.75768512, 'U': 49.76952005,
         'V': 38.31619639,'W': 35.54347121, 'X': 39.42439451, 'Y': 36.797225,  'Z': 34.14414949},
    6:  {'A': 62.22685343,'B': 37.27441235, 'C': 0.0,         'D': 48.74361854,'E': 44.8788159,  'F': 56.9879805,  'G': 45.37592309,
         'H': 48.83162846,'I': 66.98114916, 'J': 40.45838483, 'K': 31.2847248, 'L': 33.08363554, 'M': 56.25759863, 'N': 31.94630715,
         'O': 51.43007163,'P': 0.0,         'Q': 34.49614376, 'R': 45.71626622,'S': 78.7696149,  'T': 54.78981822, 'U': 42.99888813,
         'V': 62.88111571,'W': 73.71217189, 'X': 60.02105355, 'Y': 65.26809456,'Z': 62.93881472},
    7:  {'A': 26.45137724,'B': 51.46850643, 'C': 34.11217187, 'D': 45.97086653,'E': 35.67813884, 'F': 23.74010684, 'G': 23.31263625,
         'H': 36.90986023,'I': 38.88837935, 'J': 33.47668727, 'K': 37.56266998,'L': 24.28967773, 'M': 29.37769812, 'N': 38.66650296,
         'O': 49.91460251,'P': 19.30418741, 'Q': 37.82680012, 'R': 56.50316019,'S': 35.58590092, 'T': 27.37731178, 'U': 46.05368118,
         'V': 27.79607121,'W': 30.17730921, 'X': 48.82801468, 'Y': 38.0304097, 'Z': 48.18047234},
    8:  {'A': 72.01672276,'B': 62.11333106, 'C': 0.0,         'D': 68.02961542,'E': 64.65389258, 'F': 76.89137015, 'G': 70.95384465,
         'H': 69.46714131,'I': 67.7358788,  'J': 75.49636161, 'K': 0.0,        'L': 54.64085812, 'M': 75.04979358, 'N': 54.65721057,
         'O': 83.32788306,'P': 60.23771341, 'Q': 67.07059777, 'R': 69.16882196,'S': 63.26159593, 'T': 67.53923028, 'U': 54.05803601,
         'V': 0.0,        'W': 59.78685891, 'X': 74.73575491, 'Y': 74.41154458,'Z': 81.09653884},
    9:  {'A': 52.37349205,'B': 46.42235397, 'C': 41.59839299, 'D': 56.01684064,'E': 46.7520161,  'F': 0.0,         'G': 50.82941181,
         'H': 53.04368696,'I': 48.67881181, 'J': 49.95923103, 'K': 48.89505879,'L': 47.82662348, 'M': 44.35518755, 'N': 42.44007662,
         'O': 0.0,        'P': 40.06110428, 'Q': 52.45512701, 'R': 65.48316262,'S': 64.00595746, 'T': 58.22000917, 'U': 46.3743675,
         'V': 61.8443124, 'W': 47.45136575, 'X': 68.7528266,  'Y': 45.40693858,'Z': 43.54536777},
    10: {'A': 29.82266582,'B': 0.0,         'C': 0.0,         'D': 35.31178863,'E': 36.5959086,  'F': 35.50775387, 'G': 51.74951176,
         'H': 30.25569233,'I': 19.95730139, 'J': 0.0,         'K': 25.62840078,'L': 35.07622403, 'M': 36.95088358, 'N': 0.0,
         'O': 0.0,        'P': 50.5199446,  'Q': 35.91122967, 'R': 31.25366179,'S': 0.0,         'T': 29.16377866, 'U': 49.93905157,
         'V': 32.61483159,'W': 14.1159019,  'X': 17.08374125, 'Y': 23.28221467,'Z': 34.52791036},
    11: {'A': 0.0,        'B': 64.9147645,  'C': 34.3265856,  'D': 52.16548218,'E': 32.59533141, 'F': 0.0,         'G': 32.89123617,
         'H': 31.47173609,'I': 40.21465042, 'J': 54.91598853, 'K': 0.0,        'L': 44.79621102, 'M': 37.89630465, 'N': 49.92077329,
         'O': 42.61515519,'P': 34.32533318, 'Q': 38.04955164, 'R': 0.0,        'S': 47.33576136, 'T': 38.81771476, 'U': 57.51501383,
         'V': 34.3232217, 'W': 35.49816309, 'X': 51.17714558, 'Y': 47.34446321,'Z': 0.0},
    12: {'A': 45.8980471, 'B': 31.18722915, 'C': 0.0,         'D': 59.81178823,'E': 50.68200521, 'F': 0.0,         'G': 64.20212941,
         'H': 69.02862625,'I': 0.0,         'J': 57.04622433, 'K': 0.0,        'L': 59.96732082, 'M': 50.01181347, 'N': 47.92271932,
         'O': 63.42639576,'P': 53.89601449, 'Q': 0.0,         'R': 51.24080338,'S': 51.27380893, 'T': 59.13803121, 'U': 68.59842503,
         'V': 0.0,        'W': 48.25546618, 'X': 50.42144623, 'Y': 55.08407807,'Z': 49.4460391},
    13: {'A': 48.19106369,'B': 44.40628842, 'C': 0.0,         'D': 49.09581183,'E': 38.16494319, 'F': 61.10876092, 'G': 46.93123685,
         'H': 40.00351192,'I': 46.19292264, 'J': 0.0,         'K': 0.0,        'L': 0.0,         'M': 36.21742876, 'N': 52.77558343,
         'O': 0.0,        'P': 40.18995177, 'Q': 44.89707677, 'R': 46.22165805,'S': 0.0,         'T': 0.0,         'U': 38.09670627,
         'V': 0.0,        'W': 34.49983204, 'X': 69.36283648, 'Y': 39.51140596,'Z': 39.7939887},
    14: {'A': 63.35712229,'B': 49.1348466,  'C': 55.84045123, 'D': 49.92630457,'E': 47.5194743,  'F': 69.50860227, 'G': 69.65797247,
         'H': 63.25594312,'I': 45.6920564,  'J': 53.66144451, 'K': 70.17172065,'L': 65.2546536,  'M': 51.51523362, 'N': 56.00711003,
         'O': 66.83614053,'P': 46.82660308, 'Q': 48.15358126, 'R': 0.0,        'S': 50.4325614,  'T': 61.21839558, 'U': 53.58504869,
         'V': 64.83321087,'W': 51.28322114, 'X': 67.64241162, 'Y': 52.94905288,'Z': 48.57569659},
    15: {'A': 23.21661838,'B': 0.0,         'C': 0.0,         'D': 38.91152044,'E': 0.0,         'F': 26.97991769, 'G': 35.16343744,
         'H': 32.55139598,'I': 43.1580082,  'J': 0.0,         'K': 51.71349843,'L': 50.18427406, 'M': 33.09510878, 'N': 0.0,
         'O': 0.0,        'P': 30.97267369, 'Q': 34.06629576, 'R': 0.0,        'S': 0.0,         'T': 37.24057715, 'U': 38.80140373,
         'V': 30.74506657,'W': 21.07844673, 'X': 32.85783722, 'Y': 31.40224066,'Z': 44.83250392},
    16: {'A': 59.44735127,'B': 44.1748031,  'C': 32.75655016, 'D': 53.47773179,'E': 66.05813146, 'F': 54.33200857, 'G': 71.41280356,
         'H': 68.1643389, 'I': 56.13166517, 'J': 48.06678561, 'K': 0.0,        'L': 67.63642348, 'M': 57.52576344, 'N': 62.91436227,
         'O': 45.04828393,'P': 57.46619098, 'Q': 63.07103108, 'R': 0.0,        'S': 57.06449411, 'T': 63.41802569, 'U': 70.75992487,
         'V': 50.918534,  'W': 49.03549704, 'X': 52.72003173, 'Y': 48.85303491,'Z': 49.69867529},
    17: {'A': 0.0,        'B': 30.78111034, 'C': 0.0,         'D': 40.14402742,'E': 0.0,         'F': 0.0,         'G': 70.87628992,
         'H': 58.3490784, 'I': 45.87437905, 'J': 40.94313695, 'K': 52.49962545,'L': 35.46781871, 'M': 64.66442699, 'N': 54.25963543,
         'O': 62.79114956,'P': 67.03239421, 'Q': 60.75368538, 'R': 0.0,        'S': 36.83979383, 'T': 61.74769626, 'U': 53.52274017,
         'V': 48.42797225,'W': 42.69171929, 'X': 24.32763957, 'Y': 47.39839643,'Z': 36.21365656},
    18: {'A': 45.89132096,'B': 45.09351328, 'C': 61.44115687, 'D': 52.08754281,'E': 66.40272557, 'F': 48.36182221, 'G': 50.59184751,
         'H': 59.54860223,'I': 34.88736273, 'J': 54.53439811, 'K': 72.20820022,'L': 50.04389968, 'M': 57.21938676, 'N': 0.0,
         'O': 0.0,        'P': 68.01033139, 'Q': 61.49142144, 'R': 55.89264212,'S': 45.4354308,  'T': 40.36106714, 'U': 56.33412284,
         'V': 37.45537259,'W': 33.0137052,  'X': 47.72284562, 'Y': 52.97734628,'Z': 50.06416461},
    19: {'A': 51.833334,  'B': 56.98644867, 'C': 0.0,         'D': 55.32213754,'E': 45.04071136, 'F': 0.0,         'G': 35.84594525,
         'H': 42.20220319,'I': 0.0,         'J': 54.22228739, 'K': 0.0,        'L': 29.92358141, 'M': 54.79125841, 'N': 40.02949988,
         'O': 30.36077788,'P': 46.90212603, 'Q': 0.0,         'R': 44.86111818,'S': 34.78124215, 'T': 34.45135029, 'U': 30.68047923,
         'V': 0.0,        'W': 40.37534357, 'X': 37.11668926, 'Y': 50.35336019,'Z': 48.57553701},
    20: {'A': 34.50565703,'B': 66.11121943, 'C': 60.27528438, 'D': 46.36922738,'E': 35.62896721, 'F': 53.03029787, 'G': 31.99846196,
         'H': 27.13117008,'I': 55.34414956, 'J': 54.95678304, 'K': 44.6546433, 'L': 53.523722,   'M': 32.39684961, 'N': 47.19285647,
         'O': 68.88669982,'P': 31.68347191, 'Q': 42.007612,   'R': 53.97798918,'S': 46.78545015, 'T': 42.50671412, 'U': 36.34075485,
         'V': 45.5451389, 'W': 20.71014538, 'X': 44.55814377, 'Y': 39.80308324,'Z': 48.76032977},
    21: {'A': 42.07319553,'B': 56.30083081, 'C': 46.3705737,  'D': 39.39928903,'E': 36.31338687, 'F': 34.84530004, 'G': 41.95518218,
         'H': 41.2448021, 'I': 42.46803025, 'J': 36.97964901, 'K': 39.41387276,'L': 25.81641837, 'M': 50.92386553, 'N': 37.89704332,
         'O': 0.0,        'P': 46.42522442, 'Q': 47.03026417, 'R': 36.79380983,'S': 40.83768305, 'T': 41.28563206, 'U': 33.24206275,
         'V': 33.9318992, 'W': 47.08569467, 'X': 41.66878306, 'Y': 48.01041741,'Z': 52.497931},
    22: {'A': 49.72173371,'B': 50.83298741, 'C': 69.17061219, 'D': 47.74161219,'E': 43.46422358, 'F': 31.06643584, 'G': 38.59874603,
         'H': 37.95987902,'I': 35.23362492, 'J': 43.67253161, 'K': 35.10406312,'L': 0.0,         'M': 49.20829529, 'N': 35.46821707,
         'O': 0.0,        'P': 41.16216481, 'Q': 55.44097904, 'R': 33.51013822,'S': 40.76427663, 'T': 38.14529041, 'U': 23.32383843,
         'V': 29.90054828,'W': 44.07493456, 'X': 47.18495954, 'Y': 44.42840662,'Z': 48.17159753}, 
    23: {'A': 37.24038386,'B': 0.0,         'C': 0.0,         'D': 42.49630517,'E': 0.0,         'F': 50.77266556, 'G': 46.01653548,
         'H': 49.41823313,'I': 35.2570074,  'J': 0.0,         'K': 50.74975178,'L': 53.60201674, 'M': 47.70141619, 'N': 0.0,
         'O': 0.0,        'P': 61.62044265, 'Q': 57.6516194,  'R': 0.0,        'S': 0.0,         'T': 24.32100874, 'U': 38.58958009,
         'V': 24.79563745,'W': 17.17678472, 'X': 37.59932659, 'Y': 28.03575359,'Z': 13.86612202},
    24: {'A': 61.07287597,'B': 40.57205673, 'C': 0.0,         'D': 57.18356273,'E': 35.91314858, 'F': 53.68593693, 'G': 71.81677851,
         'H': 64.78335108,'I': 46.31324111, 'J': 39.02529432, 'K': 64.06776636,'L': 43.11948702, 'M': 44.94328092, 'N': 50.53346133,
         'O': 55.3846227, 'P': 38.06305387, 'Q': 54.78493124, 'R': 41.96188927,'S': 55.30956069, 'T': 58.06399592, 'U': 52.52129479,
         'V': 64.67970728,'W': 47.81032906, 'X': 59.02162101, 'Y': 49.522199,  'Z': 34.61310738},
    25: {'A': 31.95258913,'B': 74.29451295, 'C': 32.07981437, 'D': 62.45974214,'E': 47.7794358,  'F': 43.31794922, 'G': 35.12160389,
         'H': 45.43301161,'I': 52.11203162, 'J': 51.12864057, 'K': 0.0,        'L': 64.38694856, 'M': 44.04004755, 'N': 49.76398805,
         'O': 65.35981429,'P': 38.94866645, 'Q': 58.99156597, 'R': 0.0,        'S': 35.65786787, 'T': 37.99160354, 'U': 37.31398614,
         'V': 37.42146692,'W': 33.25789859, 'X': 43.62188536, 'Y': 45.13051768,'Z': 54.98051158},
    26: {'A': 49.77664266,'B': 51.49535171, 'C': 55.9164317,  'D': 41.41548008,'E': 56.29132847, 'F': 42.36998142, 'G': 48.10862861,
         'H': 53.18277312,'I': 46.29124183, 'J': 44.05541123, 'K': 40.04768227,'L': 42.34902778, 'M': 58.29959786, 'N': 38.30163433,
         'O': 44.54460948,'P': 43.09916702, 'Q': 60.12529923, 'R': 52.08412673,'S': 30.66724534, 'T': 40.03589055, 'U': 34.72779412,
         'V': 41.8824673, 'W': 47.62199468, 'X': 37.04562531, 'Y': 54.04550016,'Z': 52.93722955},
    27: {'A': 34.26467847,'B': 55.35656942, 'C': 59.51761531, 'D': 0.0,        'E': 38.68296878, 'F': 43.3931631,  'G': 27.06170034,
         'H': 26.78176805,'I': 45.24423358, 'J': 34.43817655, 'K': 0.0,        'L': 0.0,         'M': 36.95022728, 'N': 0.0,
         'O': 0.0,        'P': 35.84273691, 'Q': 57.25075379, 'R': 0.0,        'S': 32.88579174, 'T': 27.31603488, 'U': 20.72163932,
         'V': 29.63881277,'W': 23.15151998, 'X': 43.46727736, 'Y': 0.0,        'Z': 46.52222948},
    28: {'A': 40.42737935,'B': 51.66820241, 'C': 42.40801701, 'D': 38.23049652,'E': 28.79842199, 'F': 33.26626807, 'G': 25.19386059,
         'H': 34.1507348, 'I': 46.7731166,  'J': 42.36545444, 'K': 30.21619813,'L': 40.3380021,  'M': 27.20201223, 'N': 25.90265432,
         'O': 54.19074989,'P': 28.94878864, 'Q': 48.32864844, 'R': 48.16509687,'S': 48.31274901, 'T': 24.66543563, 'U': 13.40905939, 
         'V': 24.15222884,'W': 14.33703651, 'X': 32.28214244, 'Y': 21.00784375,'Z': 37.68730816},
    29: {'A': 39.52368174,'B': 0.0,         'C': 0.0,         'D': 48.7694508, 'E': 0.0,         'F': 58.60551186, 'G': 40.35835709,
         'H': 29.06617622,'I': 48.82397725, 'J': 0.0,         'K': 55.25783948,'L': 68.2479411,  'M': 41.82091873, 'N': 0.0,
         'O': 0.0,        'P': 38.16331406, 'Q': 45.43988551, 'R': 0.0,        'S': 0.0,         'T': 30.65414342, 'U': 28.34894587,
         'V': 44.44116257,'W': 16.54880216, 'X': 27.62238011, 'Y': 40.05638283,'Z': 40.51870481},
    30: {'A': 63.61050253,'B': 0.0,         'C': 0.0,         'D': 41.63833288,'E': 39.15472954, 'F': 63.66106261, 'G': 61.90667658,
         'H': 56.23538877,'I': 45.0544841,  'J': 0.0,         'K': 53.42127162,'L': 64.2270816,  'M': 49.97621123, 'N': 0.0,
         'O': 0.0,        'P': 47.36752439, 'Q': 71.36990438, 'R': 39.35983209,'S': 0.0,         'T': 62.00175987, 'U': 37.02779941,
         'V': 51.57523731,'W': 48.84033475, 'X': 49.16765004, 'Y': 49.83801354,'Z': 52.0309195},
    31: {'B': 69.28899667,'C': 0.0,         'D': 61.31450282, 'E': 40.19794217,'F': 48.52936174, 'G': 42.70146579, 'H': 35.13479673,
         'I': 65.71047992,'J': 60.05840386, 'K': 59.31398425, 'L': 69.28812642,'M': 50.15028845, 'N': 42.08722438, 'O': 70.0390869, 
         'P': 47.98505358,'Q': 54.44649705, 'R': 58.50440215, 'S': 53.09316373,'T': 53.97149098, 'U': 41.76372954, 'V': 55.96324898,
         'W': 30.01853393,'X': 48.65932658, 'Y': 53.4858192,  'Z': 42.13502479}
}

In [21]:
item_df = pd.DataFrame(items).T
user_df = pd.DataFrame(users).T

In [22]:
ind=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]
"""计算物品的相似矩阵"""
similarity_matrix = pd.DataFrame(np.zeros((len(items), len(items))), index=ind, columns=ind)

# 遍历每条物品-用户评分数据
for itemId in items:
    for otheritemId in items:
        vec_item = []         # 定义列表， 保存当前两个物品的向量值
        vec_otheritem = []
        #userRagingPairCount = 0     # 两件物品均评过分的用户数
        if itemId != otheritemId:    # 物品不同
            for userId in users:    # 遍历用户-物品评分数据
                userRatings = users[userId]    # 每条数据为该用户对所有物品的评分， 这也是个字典
                
                if itemId in userRatings and otheritemId in userRatings:   # 用户对这两个物品都评过分
                    #userRagingPairCount += 1
                    vec_item.append(userRatings[itemId])
                    vec_otheritem.append(userRatings[otheritemId])
            
            # 这里可以获得相似性矩阵(共现矩阵)
            similarity_matrix[itemId][otheritemId] = np.corrcoef(np.array(vec_item), np.array(vec_otheritem))[0][1]
            # similarity_matrix[itemId][otheritemId] = cosine_similarity(np.array(vec_item), np.array(vec_otheritem))[0][1]

In [38]:
"""得到相似的前n个物品"""
n = 2
similarity_items =similarity_matrix[31].sort_values(ascending=False)[:n].index.tolist() 
print(similarity_items)

[1, 24]


In [39]:
"""计算最终得分"""
base_score = np.mean(np.array([value for value in items[31].values()]))
weighted_scores = 0.
corr_values_sum = 0.
for item in similarity_items: 
    corr_value = similarity_matrix[31][item]           # 两个物品之间的相似性
    mean_item_score = np.mean(np.array([value for value in items[item].values()]))    # 每个物品的打分平均值
    weighted_scores += corr_value * (users['A'][item]-mean_item_score)      # 加权分数
    corr_values_sum += corr_value
final_scores = base_score + weighted_scores / corr_values_sum
print('学生A对工作31的打分: ', final_scores)
user_df.loc['A'][31] = final_scores
user_df

学生A对工作31的打分:  56.14864332488844


,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
A,55.783716,38.791029,53.060629,77.354487,46.240086,62.226853,26.451377,72.016723,52.373492,29.822666,...,49.721734,37.240384,61.072876,31.952589,49.776643,34.264678,40.427379,39.523682,63.610503,56.148643
B,53.921384,75.968958,47.335862,34.439171,31.733986,37.274412,51.468506,62.113331,46.422354,0.000000,...,50.832987,0.000000,40.572057,74.294513,51.495352,55.356569,51.668202,0.000000,0.000000,69.288997
C,0.000000,0.000000,40.379652,74.325771,60.728034,0.000000,34.112172,0.000000,41.598393,0.000000,...,69.170612,0.000000,0.000000,32.079814,55.916432,59.517615,42.408017,0.000000,0.000000,0.000000
D,64.878955,60.640720,58.784051,50.789041,40.118044,48.743619,45.970867,68.029615,56.016841,35.311789,...,47.741612,42.496305,57.183563,62.459742,41.415480,0.000000,38.230497,48.769451,41.638333,61.314503
E,39.558416,0.000000,51.739607,47.399501,58.837674,44.878816,35.678139,64.653893,46.752016,36.595909,...,43.464224,0.000000,35.913149,47.779436,56.291328,38.682969,28.798422,0.000000,39.154730,40.197942
F,52.663908,55.731268,0.000000,59.589526,50.366849,56.987980,23.740107,76.891370,0.000000,35.507754,...,31.066436,50.772666,53.685937,43.317949,42.369981,43.393163,33.266268,58.605512,63.661063,48.529362
G,52.239519,45.129264,68.829953,65.291217,60.748775,45.375923,23.312636,70.953845,50.829412,51.749512,...,38.598746,46.016535,71.816779,35.121604,48.108629,27.061700,25.193861,40.358357,61.906677,42.701466
H,48.536541,39.241096,53.395707,64.141796,55.974673,48.831628,36.909860,69.467141,53.043687,30.255692,...,37.959879,49.418233,64.783351,45.433012,53.182773,26.781768,34.150735,29.066176,56.235389,35.134797
I,59.540888,59.866708,31.684729,45.820778,44.653100,66.981149,38.888379,67.735879,48.678812,19.957301,...,35.233625,35.257007,46.313241,52.112032,46.291242,45.244234,46.773117,48.823977,45.054484,65.710480
J,61.963947,67.410912,42.542741,44.811513,43.035139,40.458385,33.476687,75.496362,49.959231,0.000000,...,43.672532,0.000000,39.025294,51.128641,44.055411,34.438177,42.365454,0.000000,0.000000,60.058404


In [17]:
similarity_matrix

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
1,0.000000,0.318775,-0.102953,-0.425111,-0.462937,0.536716,0.303017,0.432465,0.117094,0.062589,...,-0.459787,0.080840,0.681991,0.121305,-0.379076,-0.340809,0.196364,0.160599,0.153245,0.817831
2,0.318775,0.000000,-0.037120,-0.230904,-0.038381,0.357169,0.221835,0.602796,-0.168596,-0.216836,...,-0.130878,-0.009691,0.170158,0.667247,-0.165436,-0.001521,0.097790,0.093881,-0.110750,0.334166
3,-0.102953,-0.037120,0.000000,0.404766,-0.044111,-0.068151,0.039587,-0.117751,0.768891,0.258032,...,0.407137,0.137172,0.206682,-0.223799,0.054765,-0.083750,-0.438348,-0.000469,0.227206,-0.210523
4,-0.425111,-0.230904,0.404766,0.000000,0.158893,0.068963,-0.496574,-0.138039,0.388674,0.312961,...,0.676974,0.264306,-0.039961,-0.364157,0.163141,0.441356,-0.432408,0.197177,0.391875,-0.660634
5,-0.462937,-0.038381,-0.044111,0.158893,0.000000,-0.410827,-0.575344,-0.091918,-0.360497,0.221706,...,0.024447,0.415977,-0.048432,0.302645,0.124551,0.262452,-0.220861,0.228198,0.106777,-0.332900
6,0.536716,0.357169,-0.068151,0.068963,-0.410827,0.000000,0.129050,0.399381,0.135376,-0.065283,...,-0.117104,-0.111598,0.493385,0.034899,-0.153429,-0.073966,-0.095943,0.097379,0.199495,0.275167
7,0.303017,0.221835,0.039587,-0.496574,-0.575344,0.129050,0.000000,0.167908,0.107779,-0.329929,...,0.006235,-0.453808,-0.097196,0.089461,0.038379,-0.143994,0.380048,-0.421140,-0.392755,0.206001
8,0.432465,0.602796,-0.117751,-0.138039,-0.091918,0.399381,0.167908,0.000000,-0.186614,0.094303,...,-0.100954,0.018318,0.185441,0.420984,0.121868,0.010908,0.171457,-0.056045,0.100771,0.313879
9,0.117094,-0.168596,0.768891,0.388674,-0.360497,0.135376,0.107779,-0.186614,0.000000,0.091625,...,0.368763,-0.004591,0.135290,-0.293342,-0.069967,0.033246,-0.110379,-0.033449,0.149892,-0.022875
10,0.062589,-0.216836,0.258032,0.312961,0.221706,-0.065283,-0.329929,0.094303,0.091625,0.000000,...,-0.100563,0.713980,0.299751,-0.182260,0.129517,-0.037948,-0.441505,0.649990,0.795503,0.004402
